# Workspace Configuration Monitoring

This notebook demonstrates collecting workspace configuration including:
- **OAP (OneLake Access Point)** settings
- **Data access policies** and security settings
- **Git integration** status
- **Compliance and governance** settings
- **Capacity assignments**

## Use Cases
1. **Security Monitoring**: Track OAP enablement and data access policies
2. **Compliance Auditing**: Monitor workspace settings for regulatory compliance
3. **Sentinel Alerting**: Detect unauthorized workspace configuration changes
4. **Governance**: Track workspace features and capabilities across your tenant

---

## 1. Environment Setup

Load the FabricLA-Connector package and configure authentication.

In [ ]:
# Import the FabricLA-Connector framework
import sys
import os

# Add parent directory to path if running locally
if '../src' not in sys.path:
    sys.path.insert(0, os.path.abspath('../src'))

from fabricla_connector import workflows
from fabricla_connector.collectors import AccessPermissionsCollector
from fabricla_connector.config import validate_config, print_config_status
from fabricla_connector.api import get_fabric_token

print("✅ FabricLA-Connector loaded successfully")

## 2. Configuration & Validation

Validate your environment configuration before collecting data.

In [ ]:
# Validate configuration
print("🔍 Validating configuration...\n")
config_status = validate_config()

if config_status['valid']:
    print("✅ Configuration is valid!\n")
    print_config_status()
else:
    print("❌ Configuration validation failed:")
    for error in config_status['errors']:
        print(f"   - {error}")
    print("\n💡 Set up your environment variables or Key Vault integration.")

## 3. Set Target Workspace

Specify the workspace ID to monitor.

In [ ]:
# Set your workspace ID
WORKSPACE_ID = "your-workspace-id-here"  # Replace with your actual workspace ID

# Optional: Override ingestion configuration
CUSTOM_DCE_ENDPOINT = None  # Set to override default DCE
CUSTOM_DCR_ID = None        # Set to override default DCR
CUSTOM_STREAM_NAME = "Custom-FabricWorkspaceConfig_CL"  # DCR stream name

print(f"🎯 Target Workspace: {WORKSPACE_ID}")
print(f"📊 Stream Name: {CUSTOM_STREAM_NAME}")

## 4. Collect Workspace Configuration (Preview)

Preview workspace configuration data before ingestion.

In [ ]:
import pandas as pd
from datetime import datetime

# Create collector instance
collector = AccessPermissionsCollector(WORKSPACE_ID)

# Collect workspace configuration
print("🔍 Collecting workspace configuration...\n")
workspace_configs = list(collector.collect_workspace_config())

if workspace_configs:
    print(f"✅ Collected {len(workspace_configs)} workspace configuration record(s)\n")
    
    # Display as DataFrame for better readability
    df = pd.DataFrame(workspace_configs)
    
    # Display key configuration fields
    print("📋 Workspace Configuration Summary:")
    print("=" * 80)
    
    if len(workspace_configs) > 0:
        config = workspace_configs[0]
        
        print(f"Workspace Name: {config.get('WorkspaceName', 'N/A')}")
        print(f"Workspace Type: {config.get('WorkspaceType', 'N/A')}")
        print(f"State: {config.get('State', 'N/A')}")
        print(f"Capacity ID: {config.get('CapacityId', 'N/A')}")
        print()
        print("🔐 OneLake Access Point (OAP) Configuration:")
        print(f"   - OneLake Access Enabled: {config.get('OneLakeAccessEnabled', False)}")
        print(f"   - OAP Enabled: {config.get('OneLakeAccessPointEnabled', False)}")
        print()
        print("🔒 Security Settings:")
        print(f"   - Public Internet Access: {config.get('PublicInternetAccess', 'N/A')}")
        print(f"   - Read Only State: {config.get('ReadOnlyState', 'N/A')}")
        print(f"   - Managed Virtual Network: {config.get('ManagedVirtualNetwork', 'N/A')}")
        print()
        print("🔧 Features:")
        print(f"   - Git Enabled: {config.get('GitEnabled', False)}")
        print(f"   - Git Repository: {config.get('GitRepositoryUrl', 'N/A')}")
        print(f"   - On Dedicated Capacity: {config.get('IsOnDedicatedCapacity', False)}")
        print()
        print("📊 Compliance:")
        print(f"   - Data Classification: {config.get('DataClassification', 'N/A')}")
        print(f"   - Sensitivity Label: {config.get('SensitivityLabel', 'N/A')}")
        print()
    
    # Display full DataFrame
    print("\n📊 Full Configuration Data:")
    print("=" * 80)
    display(df)
    
else:
    print("⚠️ No workspace configuration data collected.")
    print("💡 This may require admin permissions. Check authentication and permissions.")

## 5. Collect and Ingest to Log Analytics

Collect workspace configuration and ingest to Azure Monitor Log Analytics.

In [ ]:
# Collect and ingest workspace configuration
print("🚀 Starting workspace configuration collection and ingestion...\n")

results = workflows.collect_and_ingest_workspace_config(
    workspace_id=WORKSPACE_ID,
    dce_endpoint=CUSTOM_DCE_ENDPOINT,
    dcr_immutable_id=CUSTOM_DCR_ID,
    stream_name=CUSTOM_STREAM_NAME
)

# Display results
print("\n" + "=" * 80)
print("📊 WORKSPACE CONFIGURATION COLLECTION RESULTS")
print("=" * 80)
print(f"\n✅ Workspace Config:")
print(f"   - Collected: {results['workspace_config']['collected']}")
print(f"   - Ingested: {results['workspace_config']['ingested']}")

if results['errors']:
    print(f"\n⚠️ Errors encountered: {len(results['errors'])}")
    for error in results['errors']:
        print(f"   - {error}")
else:
    print("\n✅ No errors encountered!")

## 6. Query Log Analytics Data

Example KQL queries to analyze workspace configuration in Log Analytics.

### Query 1: Check OAP Enablement Status

```kql
Custom_FabricWorkspaceConfig_CL
| where TimeGenerated > ago(1d)
| project 
    TimeGenerated,
    WorkspaceName,
    WorkspaceId,
    OneLakeAccessEnabled,
    OneLakeAccessPointEnabled,
    PublicInternetAccess,
    IsOnDedicatedCapacity
| order by TimeGenerated desc
```

### Query 2: Find Workspaces with OAP Enabled

```kql
Custom_FabricWorkspaceConfig_CL
| where TimeGenerated > ago(7d)
| where OneLakeAccessPointEnabled == true
| summarize 
    LastSeen = max(TimeGenerated),
    CapacityCount = dcount(CapacityId)
    by WorkspaceName, WorkspaceId, PublicInternetAccess
| order by LastSeen desc
```

### Query 3: Track Configuration Changes Over Time

```kql
Custom_FabricWorkspaceConfig_CL
| where WorkspaceId == "your-workspace-id"
| project 
    TimeGenerated,
    OneLakeAccessEnabled,
    GitEnabled,
    PublicInternetAccess,
    DataClassification,
    IsOnDedicatedCapacity
| order by TimeGenerated asc
```

### Query 4: Security Compliance Check

```kql
Custom_FabricWorkspaceConfig_CL
| where TimeGenerated > ago(1d)
| where PublicInternetAccess == true or OneLakeAccessEnabled == true
| project 
    WorkspaceName,
    PublicInternetAccess,
    OneLakeAccessEnabled,
    OneLakeAccessPointEnabled,
    ManagedVirtualNetwork,
    SensitivityLabel
| summarize count() by WorkspaceName, PublicInternetAccess
```

## 7. Microsoft Sentinel Alert Rules

Example Sentinel analytic rules for workspace configuration monitoring.

### Alert 1: Unauthorized OAP Enablement

Detect when OneLake Access Point is enabled on a workspace.

```kql
let Baseline = Custom_FabricWorkspaceConfig_CL
| where TimeGenerated > ago(7d) and TimeGenerated < ago(1d)
| where OneLakeAccessPointEnabled == false
| distinct WorkspaceId;

Custom_FabricWorkspaceConfig_CL
| where TimeGenerated > ago(1h)
| where OneLakeAccessPointEnabled == true
| where WorkspaceId in (Baseline)
| project 
    TimeGenerated,
    WorkspaceName,
    WorkspaceId,
    OneLakeAccessEnabled,
    OneLakeAccessPointEnabled,
    PublicInternetAccess,
    AlertSeverity = "High",
    AlertDescription = "OAP was enabled on a workspace that previously had it disabled"
```

### Alert 2: Public Internet Access Enabled

```kql
Custom_FabricWorkspaceConfig_CL
| where TimeGenerated > ago(1h)
| where PublicInternetAccess == true
| where OneLakeAccessEnabled == true
| project 
    TimeGenerated,
    WorkspaceName,
    WorkspaceId,
    PublicInternetAccess,
    OneLakeAccessPointEnabled,
    SensitivityLabel,
    AlertSeverity = "Medium",
    AlertDescription = "Workspace has both Public Internet Access and OneLake Access enabled"
```

### Alert 3: Missing Sensitivity Label

```kql
Custom_FabricWorkspaceConfig_CL
| where TimeGenerated > ago(24h)
| where isempty(SensitivityLabel) or SensitivityLabel == "" or SensitivityLabel == "N/A"
| where OneLakeAccessEnabled == true or PublicInternetAccess == true
| project 
    TimeGenerated,
    WorkspaceName,
    WorkspaceId,
    OneLakeAccessEnabled,
    PublicInternetAccess,
    AlertSeverity = "Low",
    AlertDescription = "Workspace with data access features enabled but no sensitivity label"
```

## 8. Combine with User Activity for Complete Story

Correlate workspace configuration with user activity for comprehensive monitoring.

In [ ]:
# Example: Collect workspace config + user activity + permissions together
print("🚀 Comprehensive Security Monitoring\n")

# 1. Workspace Configuration (including OAP)
config_results = workflows.collect_and_ingest_workspace_config(
    workspace_id=WORKSPACE_ID,
    stream_name="Custom-FabricWorkspaceConfig_CL"
)

# 2. Access Permissions (user roles)
perm_results = workflows.collect_and_ingest_access_permissions(
    workspace_id=WORKSPACE_ID
)

# 3. User Activity (actions taken)
activity_results = workflows.collect_and_ingest_user_activity(
    workspace_id=WORKSPACE_ID,
    lookback_hours=24
)

print("\n" + "=" * 80)
print("📊 COMPREHENSIVE SECURITY MONITORING RESULTS")
print("=" * 80)
print(f"\n✅ Workspace Config:")
print(f"   - Collected: {config_results['workspace_config']['collected']}")
print(f"   - Ingested: {config_results['workspace_config']['ingested']}")
print(f"\n✅ Permissions:")
print(f"   - Workspace: {perm_results['workspace_permissions']['collected']} collected")
print(f"   - Items: {perm_results['item_permissions']['collected']} collected")
print(f"\n✅ User Activity:")
print(f"   - Collected: {activity_results['collected']}")
print(f"   - Ingested: {activity_results['ingested']}")

## 9. Advanced KQL: Correlate Config + Activity + Permissions

Complete user story detection combining all three log sources.

### Detect: Admin Creates Notebook → OAP Workspace → External API Call

```kql
// Step 1: Get OAP-enabled workspaces
let OAPWorkspaces = Custom_FabricWorkspaceConfig_CL
| where TimeGenerated > ago(7d)
| where OneLakeAccessPointEnabled == true
| distinct WorkspaceId, WorkspaceName;

// Step 2: Get admin users
let AdminUsers = Custom_FabricPermissions_CL
| where TimeGenerated > ago(7d)
| where Role == "Admin"
| distinct PrincipalId, WorkspaceId;

// Step 3: Get notebook creation events
let NotebookCreations = Custom_FabricActivity_CL
| where TimeGenerated > ago(24h)
| where ActivityType == "CreateNotebook"
| where UserId in ((AdminUsers | project PrincipalId))
| where WorkspaceId in ((OAPWorkspaces | project WorkspaceId))
| project NotebookCreated=TimeGenerated, UserId, WorkspaceId, ItemName;

// Step 4: Get Spark logs with outbound calls
let OutboundCalls = Custom_SparkLogs_CL
| where TimeGenerated > ago(24h)
| where LogMessage contains "googleapis.com" or LogMessage contains "outbound"
| project NetworkCall=TimeGenerated, SessionId, WorkspaceId, LogMessage;

// Step 5: Correlate the events
NotebookCreations
| join kind=inner (OutboundCalls) on WorkspaceId
| where NetworkCall > NotebookCreated and (NetworkCall - NotebookCreated) < 4h
| join kind=inner (OAPWorkspaces) on WorkspaceId
| project 
    UserId,
    WorkspaceName,
    NotebookCreated,
    NetworkCall,
    ItemName,
    LogMessage,
    AlertSeverity = "High",
    AlertDescription = "Admin created notebook in OAP-enabled workspace with external API call"
```

## 10. Next Steps

1. **Schedule this notebook** to run periodically (e.g., hourly) to track workspace configurations
2. **Create DCR** for `Custom-FabricWorkspaceConfig_CL` table in Log Analytics
3. **Set up Sentinel alerts** using the KQL queries above
4. **Integrate with existing monitoring** workflows for comprehensive coverage

### Required DCR Schema Fields:
- TimeGenerated (datetime)
- WorkspaceId (string)
- WorkspaceName (string)
- OneLakeAccessEnabled (boolean)
- OneLakeAccessPointEnabled (boolean)
- PublicInternetAccess (string)
- GitEnabled (boolean)
- IsOnDedicatedCapacity (boolean)
- DataClassification (string)
- SensitivityLabel (string)

---

**Documentation:**
- [Azure Monitor Logs Ingestion API](https://learn.microsoft.com/azure/azure-monitor/logs/logs-ingestion-api-overview)
- [Microsoft Fabric Admin APIs](https://learn.microsoft.com/rest/api/fabric/admin)
- [Microsoft Sentinel Analytics Rules](https://learn.microsoft.com/azure/sentinel/detect-threats-built-in)